In [3]:
import pandas as pd
from pathlib import Path
import re
import os
import shutil
import json

In [4]:
nifti_folder = Path("nifti")
excel_folder = Path("raw_reports")

excel_files = list(excel_folder.glob("*.xlsx"))

if not excel_files:
    print(f"No Excel files found in {excel_folder.resolve()}")
    exit(1)

print("Select an Excel file from the list:")
for i, file_path in enumerate(excel_files, 1):
    print(f"{i}: {file_path.name}")
while True:
    choice = input(f"Enter the number (1-{len(excel_files)}): ").strip()
    if choice.isdigit():
        idx = int(choice)
        if 1 <= idx <= len(excel_files):
            excel_path = excel_files[idx - 1]
            break
    print("Invalid choice, please try again.")

print(f"Excel file selected: {excel_path}")

Select an Excel file from the list:
1: 20230831-20240831.xlsx
Excel file selected: raw_reports/20230831-20240831.xlsx


In [5]:
df = pd.read_excel(excel_path, dtype=str)
df['N° IPP'] = df['N° IPP'].str.extract(r'(\d+)') # Remove any extra characters
df['Date'] = pd.to_datetime(df['Date prévue'], dayfirst=True, errors='coerce')
df['Date'] = df['Date'].dt.strftime('%Y%m%d')

/tmp/ipykernel_340747/2343179547.py:3: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date prévue'], dayfirst=True, errors='coerce')


In [ ]:
# Remove absent nifti / excel pairs
valid_pairs = set(zip(df['N° IPP'], df['Date']))

nii_files = list(nifti_folder.rglob("*.nii.gz"))
file_map = {}  # (ipp, date) -> full file path(s)
for f in nii_files:
    base = f.name[:-7]
    match = re.match(r"(\d+)_([0-9]{8})_.*", base)
    if not match:
        print(f"Skipping malformed file: {f.name}")
        continue
    ipp, date = match.groups()
    file_map.setdefault((ipp, date), []).append(f)

file_pairs = set(file_map.keys())

df_filtered = df[df.apply(lambda row: (row['N° IPP'], row['Date']) in file_pairs, axis=1)]

unmatched_files = [f for (ipp, date), paths in file_map.items()
                   if (ipp, date) not in valid_pairs
                   for f in paths]

for f in unmatched_files:
    print(f"Deleting unmatched file: {f}")
    os.remove(f)

In [ ]:
# Check or create IPP -> ID lookup table
lookup_df = df_filtered[['N° IPP']].copy()
lookup_df = lookup_df.rename(columns={"N° IPP": "IPP"})

lookup_path = Path("lookup_table.xlsx")

if lookup_path.exists():
    existing_df = pd.read_excel(lookup_path)
    combined_df = pd.concat([existing_df, lookup_df], ignore_index=True)
else:
    combined_df = lookup_df

combined_df = combined_df.drop_duplicates(subset=["IPP"], keep="first")


if 'SubjectID' not in combined_df.columns:
    combined_df['SubjectID'] = None

missing_mask = combined_df['SubjectID'].isna() | (combined_df['SubjectID'] == "")

existing_ids = combined_df['SubjectID'].dropna().unique()
existing_numbers = [int(sid.replace("SUB", "")) for sid in existing_ids if sid.startswith("SUB") and sid[3:].isdigit()]
next_number = max(existing_numbers, default=0) + 1
num_missing = missing_mask.sum()
new_ids = [f"SUB{next_number + i:06d}" for i in range(num_missing)]
combined_df.loc[missing_mask, 'SubjectID'] = new_ids

combined_df.to_excel(lookup_path, index=False)
print(f"Lookup table saved to {lookup_path}")


df_filtered = df_filtered.merge(
    combined_df[['IPP', 'SubjectID']],
    how='left',
    left_on='N° IPP',
    right_on='IPP'
)

df_filtered = df_filtered.drop(columns=['IPP'])

output_excel = excel_path.with_name(excel_path.stem + "_filtered.xlsx")
df_filtered.to_excel(output_excel, index=False)
print(f"Filtered Excel saved to {output_excel}")

In [ ]:
# Move to dataset folder
output_base = Path("dataset")
output_base.mkdir(exist_ok=True)

ipp_date_to_subj = {
    (row['N° IPP'], row['Date']): row['SubjectID']
    for _, row in df_filtered.iterrows()
}

for (ipp, date), files in file_map.items():
    subject_id = ipp_date_to_subj.get((ipp, date))
    if not subject_id:
        continue

    for filepath in files:
        # Extract the part after the second underscore for filename
        base = filepath.name[:-7]  # Remove '.nii.gz'
        parts = base.split('_', 2)  # Split max 3 parts
        if len(parts) < 3:
            print(f"Warning: filename '{filepath.name}' has fewer than 3 parts, skipping")
            continue
        rest_filename = parts[2] + ".nii.gz"

        # Construct new path: dataset/SubjectID/date/rest_filename
        new_dir = output_base / subject_id / date
        new_dir.mkdir(parents=True, exist_ok=True)

        new_path = new_dir / rest_filename

        print(f"Moving {filepath} -> {new_path}")
        shutil.move(str(filepath), str(new_path))

shutil.rmtree(nifti_folder)

In [ ]:
def clean_text(x):
    if pd.isna(x):
        return ""
    return str(x).replace('\n', ' ').replace('\r', ' ').strip()

json_path = "dataset/reports.json"

# Prepare new data
json_data = []
for _, row in df_filtered.iterrows():
    record = {
        "SubjectID": clean_text(row["SubjectID"]),
        "Date": clean_text(row["Date"]),
        "Indication": clean_text(row.get("Indication")),
        "PET_results": clean_text(row.get("Resultat_TEP")),
        "MRI_results": clean_text(row.get("Resultat_IRM")),
        "Conclusion": clean_text(row.get("Conclusion"))
    }
    json_data.append(record)

# Load existing data if file exists
if os.path.exists(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        try:
            existing_data = json.load(f)
            if not isinstance(existing_data, list):
                existing_data = []
        except json.JSONDecodeError:
            existing_data = []
else:
    existing_data = []

# Append new data
combined_data = existing_data + json_data

# Save back to JSON
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(combined_data, f, indent=2, ensure_ascii=False)

print(f"JSON data appended to {json_path}")